In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
system_name = "Wannon Water"
location = "Australia"
base_url = "https://pvoutput.org/list.jsp?p={}&id=57479&sid=63726&gs=0&v=0&o=date&d=desc"
pages_to_scrape = 5

In [3]:
options = Options()
options.binary_location = r"C:\Program Files\chrome for testing\chrome-win64\chrome.exe"
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
service = Service(executable_path=r"C:\Program Files\chrome for testing\chrome-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [4]:
all_data = []

for page in range(pages_to_scrape):
    url = base_url.format(page)
    print(f"📄 Scraping page {page + 1} from: {url}")
    
    try:
        driver.get(url)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="tbl_main")

        if not table:
            print(f"⚠️ No table found on page {page + 1}. Retrying once...")
            time.sleep(5)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", id="tbl_main")

        if table:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
                if len(cols) >= 9:
                    cols.append(system_name)
                    cols.append(location)
                    all_data.append(cols)
        else:
            print(f"❌ Failed again: No table found on page {page + 1}")
    except Exception as e:
        print(f"🚨 Error scraping page {page + 1}: {e}")

driver.quit()

📄 Scraping page 1 from: https://pvoutput.org/list.jsp?p=0&id=57479&sid=63726&gs=0&v=0&o=date&d=desc
📄 Scraping page 2 from: https://pvoutput.org/list.jsp?p=1&id=57479&sid=63726&gs=0&v=0&o=date&d=desc
📄 Scraping page 3 from: https://pvoutput.org/list.jsp?p=2&id=57479&sid=63726&gs=0&v=0&o=date&d=desc
📄 Scraping page 4 from: https://pvoutput.org/list.jsp?p=3&id=57479&sid=63726&gs=0&v=0&o=date&d=desc
📄 Scraping page 5 from: https://pvoutput.org/list.jsp?p=4&id=57479&sid=63726&gs=0&v=0&o=date&d=desc


In [5]:
header = [
    "Date", "Generated", "Efficiency", "Exported", "Peak Power",
    "Peak Time", "Conditions", "Temperature", "Comments",
    "System Name", "Location"
]

In [6]:
df = pd.DataFrame(all_data, columns=header)
df.drop(columns=["Exported", "Temperature", "Comments", "Peak Power", "Peak Time", "Conditions"], inplace=True)
print(df.head())

       Date    Generated   Efficiency   System Name   Location
0  31/03/25     0.000kWh  0.000kWh/kW  Wannon Water  Australia
1  30/03/25   909.711kWh  3.600kWh/kW  Wannon Water  Australia
2  29/03/25   622.247kWh  2.462kWh/kW  Wannon Water  Australia
3  28/03/25  1125.239kWh  4.453kWh/kW  Wannon Water  Australia
4  27/03/25  1427.867kWh  5.650kWh/kW  Wannon Water  Australia


In [7]:
page6 = pd.read_csv("WW_page6.csv")
page7 = pd.read_csv("WW_page7.csv")
page8 = pd.read_csv("WW_page8.csv")
page9 = pd.read_csv("WW_page9.csv")
page10 = pd.read_csv("WW_page10.csv")

In [8]:
page6["System Name"] = "Wannon Water"
page6["Location"] = "Australia"
page7["System Name"] = "Wannon Water"
page7["Location"] = "Australia"
page8["System Name"] = "Wannon Water"
page8["Location"] = "Australia"
page9["System Name"] = "Wannon Water"
page9["Location"] = "Australia"
page10["System Name"] = "Wannon Water"
page10["Location"] = "Australia"

In [9]:
master_df = pd.concat([df, page6, page7, page8, page9, page10], ignore_index=True)

In [10]:
master_df

,Date,Generated,Efficiency,System Name,Location
0,31/03/25,0.000kWh,0.000kWh/kW,Wannon Water,Australia
1,30/03/25,909.711kWh,3.600kWh/kW,Wannon Water,Australia
2,29/03/25,622.247kWh,2.462kWh/kW,Wannon Water,Australia
3,28/03/25,1125.239kWh,4.453kWh/kW,Wannon Water,Australia
4,27/03/25,1427.867kWh,5.650kWh/kW,Wannon Water,Australia
...,...,...,...,...,...
395,3/1/2024,1617.257,6.399403,Wannon Water,Australia
396,2/29/2024,1250.759,4.949189,Wannon Water,Australia
397,2/28/2024,711.763,2.816409,Wannon Water,Australia
398,2/27/2024,1665.504,6.590313,Wannon Water,Australia


In [11]:
master_df["System Size (kW)"] = 252.720

In [12]:
master_df.to_csv("daily_Wannon Water_master.csv", index=False)